Будем осуществлять работу с непростым набором данных “horse_data.csv” о состоянии здоровья лошадей, испытывающих кишечные колики. Цель – максимально корректно заполнить пропуски.

Задание 1. Загрузка данных
Изучить представленный набор данных на основе описания его столбцов в файле “horse_data.names” , загрузить его и оставить 8 столбцов для дальнейшего изучения: surgery?, Age, rectal temperature, pulse, respiratory rate, temperature of extremities, pain, outcome.

Задание 2. Первичное изучение данных
Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

Задание 3. Работа с пропусками
Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.

In [160]:
import pandas as pd
import numpy as np

In [159]:
import seaborn as sns
import matplotlib.pyplot as plt

In [158]:
"""Чтение данных"""
horse_data = pd.read_csv('/Users/sofagusina/Desktop/программирование/machine_learning/статистика/statictics/hw2/Файлы_для_ДЗ/horse_data.csv',usecols=[0,1,3,4,5,6,7,11,23],names=['surgery?','age','rectal.temp','pulse','resp.rate','temp.of extremites','perh.pulse','pain','outcome'],header=0)
horse_data.head()
horse_data = horse_data.replace('?', np.NaN).apply(pd.to_numeric)
horse_data['age'] = horse_data['age'].replace(9,2)
horse_data.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   surgery?            298 non-null    float64
 1   age                 299 non-null    int64  
 2   rectal.temp         239 non-null    float64
 3   pulse               275 non-null    float64
 4   resp.rate           241 non-null    float64
 5   temp.of extremites  243 non-null    float64
 6   perh.pulse          230 non-null    float64
 7   pain                255 non-null    float64
 8   outcome             299 non-null    int64  
dtypes: float64(7), int64(2)
memory usage: 21.2 KB


In [157]:
"""Анализ непрерывных данных""" 
"""Для столбцов с непрерывными данными - ректальная температура, пульс, частота дыхания -  можно определить основные статистические характеристики и расчитать выбросы"""
columns_number = ['rectal.temp','pulse','resp.rate']
for column in columns_number:
    iqr_rectal = horse_data[column].quantile(0.75) - horse_data[column].quantile(0.25)
    lower_bound = horse_data[column].quantile(0.25) - 1.5 * iqr_rectal
    upper_bound = horse_data[column].quantile(0.75) + 1.5 * iqr_rectal
    outliers = []
    for el in horse_data[column]:
        if el < lower_bound or el > upper_bound:
            outliers.append(el)
    print(f"Outliers in column {column}:", outliers)
horse_data.describe()

Outliers in column rectal.temp: [39.9, 35.4, 40.3, 39.7, 36.4, 40.3, 39.6, 36.8, 39.5, 36.5, 36.0, 39.5, 39.5, 36.1, 36.6, 40.8, 36.9, 40.0, 39.5, 36.5]
Outliers in column pulse: [164.0, 150.0, 160.0, 184.0, 150.0]
Outliers in column resp.rate: [84.0, 96.0, 72.0, 80.0, 80.0, 68.0, 96.0, 66.0, 68.0, 90.0, 80.0, 70.0, 88.0, 84.0, 68.0, 90.0, 70.0]


,surgery?,age,rectal.temp,pulse,resp.rate,temp.of extremites,perh.pulse,pain,outcome
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000
mean,1.394649,1.080268,38.173244,71.404682,29.515050,2.441472,1.839465,2.926421,1.361204
std,0.489595,0.272162,0.655657,28.013422,16.216518,0.989417,1.020420,0.901865,0.481155
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,37.900000,48.000000,20.000000,1.000000,1.000000,3.000000,1.000000
50%,1.000000,1.000000,38.200000,60.000000,24.000000,3.000000,1.000000,3.000000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,3.000000,3.000000,2.000000
max,2.000000,2.000000,40.800000,184.000000,96.000000,4.000000,4.000000,4.000000,2.000000


In [156]:
"""Анализ категориальных данных"""
columns_category = ['surgery?','age','temp.of extremites','perh.pulse','pain','outcome']
"""Для столбцов с категориальными данными можно расчитать моду, распределение данных и уникальные значения на проверку выбросов"""

for column in columns_category:
    unique_value_list = []
    mode = horse_data[column].mode()[0]
    count = horse_data[column].value_counts(dropna=False,sort=True)
    unique_value = horse_data[column].unique()
    unique_value_list.append(unique_value)
    print (f"Основные характеристики столбца {column}: мода - {mode}, количество данных - {count} уникальные значения", unique_value_list)
"""В ходе анализа нет конкретных ошибок в данных, кроме пропусков"""


Основные характеристики столбца surgery?: мода - 1.0, количество данных - surgery?
1.0    181
2.0    118
Name: count, dtype: int64 уникальные значения [array([1., 2.])]
Основные характеристики столбца age: мода - 1, количество данных - age
1    275
2     24
Name: count, dtype: int64 уникальные значения [array([1, 2])]
Основные характеристики столбца temp.of extremites: мода - 3.0, количество данных - temp.of extremites
3.0    160
1.0     82
2.0     30
4.0     27
Name: count, dtype: int64 уникальные значения [array([3., 1., 4., 2.])]
Основные характеристики столбца perh.pulse: мода - 1.0, количество данных - perh.pulse
1.0    175
3.0    111
4.0      8
2.0      5
Name: count, dtype: int64 уникальные значения [array([3., 1., 4., 2.])]
Основные характеристики столбца pain: мода - 3.0, количество данных - pain
3.0    172
4.0     72
1.0     39
2.0     16
Name: count, dtype: int64 уникальные значения [array([4., 3., 1., 2.])]
Основные характеристики столбца outcome: мода - 1, количество данны

'В ходе анализа нет конкретных ошибок в данных, кроме пропусков'

Для категориальных данных лучшим способом замены пропусков является мода.

Для количественных данных, если в непрерывной величине есть выбросы, то стоит выбрать для замены медиану, если в непрерывной величине нет выбросов, то стоит выбрать среднее.
Заполнение пропусков на основе других признаков, категорий и группировок может быть точнее, чем просто замена пропущенных значений на модальные, средние или медианные значения по конкретным столбцам.



In [155]:
"""Определение взаимосвязи величин"""
# plt.figure(figsize=[8,5])
# horse_data.groupby('surgery?')['pain'].mean().plot(kind="bar")
# plt.title("Распределение уровня боли от наличия операции")
# plt.show()


'Определение взаимосвязи величин'

In [154]:
"""Замена пропусков в категориальных данных"""
"""Для замены пропусков в столбце температура конечностей сгруппируем по столбцу боли и заменим пропуски модой группы (есть взаимосвязь)"""
"""Для замены пропусков в столбце периферийного пульса также сгруппируем по боли и заменим модой групп (есть взаимосвязь)"""
"""Для замены пропусков в столбце боль сгруппируем по наличии операции и заполним модой групп (есть взаимосвязь)"""
"""Для замены пропусков в столбце возраст применим моду"""

def mode_or_nan(x): ## Проверка на наличие мод
    mode_vals = x.mode()
    return mode_vals.iloc[0] if not mode_vals.empty else np.nan
horse_data['surgery?'] = horse_data['surgery?'].fillna(horse_data['surgery?'].mode()[0])
horse_data['pain'] = horse_data['pain'].fillna(horse_data.groupby('surgery?')['pain'].transform(mode_or_nan))
horse_data['temp.of extremites'] = horse_data['temp.of extremites'].fillna(horse_data.groupby('pain')['temp.of extremites'].transform(mode_or_nan))
horse_data['perh.pulse'] = horse_data['perh.pulse'].fillna(horse_data.groupby('pain')['perh.pulse'].transform(mode_or_nan))
horse_data['age'] = horse_data['age'].fillna(horse_data['age'].mode()[0])


In [153]:
"""Замена пропусков в непрерывынх данных"""
"""Для замены пропусков в столбце ректальная температура воспользуемся медианой (нет зависимостей между температурой и возрастом/боль/исходом/наличием операции (проверено по диаграмме))"""
"""Для замены пропусков в столбце пульс сгруппируем данные по столбцу возраст и заполним пропуски медианой группы (есть зависимость)"""
"""Для замены пропусков в столбце частота дыхания сгруппируем данные по столбцу возраст и заполним пропуски медианой группы (есть зависимость)"""
horse_data['rectal.temp'] = horse_data['rectal.temp'].fillna(horse_data['rectal.temp'].median())
horse_data['pulse'] = horse_data['pulse'].fillna(horse_data.groupby('age')['pulse'].transform('median'))
horse_data['resp.rate'] = horse_data['resp.rate'].fillna(horse_data.groupby('age')['resp.rate'].transform('median'))
horse_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   surgery?            299 non-null    float64
 1   age                 299 non-null    int64  
 2   rectal.temp         299 non-null    float64
 3   pulse               299 non-null    float64
 4   resp.rate           299 non-null    float64
 5   temp.of extremites  299 non-null    float64
 6   perh.pulse          299 non-null    float64
 7   pain                299 non-null    float64
 8   outcome             299 non-null    int64  
dtypes: float64(7), int64(2)
memory usage: 21.2 KB


In [152]:
"""Анализ категориальных данных после замены пропусков"""
columns_category = ['surgery?','age','temp.of extremites','perh.pulse','pain','outcome']
for column in columns_category:
    unique_value_list = []
    mode = horse_data[column].mode()[0]
    count = horse_data[column].value_counts(dropna=False,sort=True)
    unique_value = horse_data[column].unique()
    unique_value_list.append(unique_value)
    print (f"Основные характеристики столбца {column}: мода - {mode}, количество данных - {count} уникальные значения", unique_value_list)


Основные характеристики столбца surgery?: мода - 1.0, количество данных - surgery?
1.0    181
2.0    118
Name: count, dtype: int64 уникальные значения [array([1., 2.])]
Основные характеристики столбца age: мода - 1, количество данных - age
1    275
2     24
Name: count, dtype: int64 уникальные значения [array([1, 2])]
Основные характеристики столбца temp.of extremites: мода - 3.0, количество данных - temp.of extremites
3.0    160
1.0     82
2.0     30
4.0     27
Name: count, dtype: int64 уникальные значения [array([3., 1., 4., 2.])]
Основные характеристики столбца perh.pulse: мода - 1.0, количество данных - perh.pulse
1.0    175
3.0    111
4.0      8
2.0      5
Name: count, dtype: int64 уникальные значения [array([3., 1., 4., 2.])]
Основные характеристики столбца pain: мода - 3.0, количество данных - pain
3.0    172
4.0     72
1.0     39
2.0     16
Name: count, dtype: int64 уникальные значения [array([4., 3., 1., 2.])]
Основные характеристики столбца outcome: мода - 1, количество данны

In [151]:
"""Анализ непрерывных данных после замены пропусков""" 
columns_number = ['rectal.temp','pulse','resp.rate']

for column in columns_number:
    iqr_rectal = horse_data[column].quantile(0.75) - horse_data[column].quantile(0.25)
    lower_bound = horse_data[column].quantile(0.25) - 1.5 * iqr_rectal
    upper_bound = horse_data[column].quantile(0.75) + 1.5 * iqr_rectal
    outliers = []
    for el in horse_data[column]:
        if el < lower_bound or el > upper_bound:
            outliers.append(el)
    print(f"Outliers in column {column}:", outliers)

horse_data.describe()

Outliers in column rectal.temp: [39.9, 35.4, 40.3, 39.7, 36.4, 40.3, 39.6, 36.8, 39.5, 36.5, 36.0, 39.5, 39.5, 36.1, 36.6, 40.8, 36.9, 40.0, 39.5, 36.5]
Outliers in column pulse: [164.0, 150.0, 160.0, 184.0, 150.0]
Outliers in column resp.rate: [84.0, 96.0, 72.0, 80.0, 80.0, 68.0, 96.0, 66.0, 68.0, 90.0, 80.0, 70.0, 88.0, 84.0, 68.0, 90.0, 70.0]


,surgery?,age,rectal.temp,pulse,resp.rate,temp.of extremites,perh.pulse,pain,outcome
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000
mean,1.394649,1.080268,38.173244,71.404682,29.515050,2.441472,1.839465,2.926421,1.361204
std,0.489595,0.272162,0.655657,28.013422,16.216518,0.989417,1.020420,0.901865,0.481155
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,37.900000,48.000000,20.000000,1.000000,1.000000,3.000000,1.000000
50%,1.000000,1.000000,38.200000,60.000000,24.000000,3.000000,1.000000,3.000000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,3.000000,3.000000,2.000000
max,2.000000,2.000000,40.800000,184.000000,96.000000,4.000000,4.000000,4.000000,2.000000


Вывод: строки с пропусками не были удалены, пропуски были заменены конкретными данными - медианой группы, модой группы, медианой или модой дата-фрейма. Сильных изменений статистических показателй выявлено не было